<a href="https://colab.research.google.com/github/polyamuscat/ColabLabs/blob/main/Exercise4-Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ниже приведен код со ссылкой на набор данных «happy» или «sad», содержащий 80 изображений, 40 счастливых и 40 грустных.
Создайте сверточную нейронную сеть, которая обучается со 100% точностью на этих изображениях, с отменой обучения при достижении точности обучения .999

In [1]:
import tensorflow as tf
import os
import zipfile
#
!wget --no-check-certificate "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
   -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()


--2021-05-28 05:51:09--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.193.128, 172.217.204.128, 172.217.203.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.03s   

2021-05-28 05:51:10 (95.3 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [2]:
import os

train_happy_dir = os.path.join('/tmp/h-or-s/happy')

train_sad_dir = os.path.join('/tmp/h-or-s/sad')

train_happy_names = os.listdir(train_happy_dir)
print(train_happy_names[:10])

train_sad_names = os.listdir(train_sad_dir)
print(train_sad_names[:10])

['happy2-00.png', 'happy2-08.png', 'happy2-18.png', 'happy2-01.png', 'happy1-05.png', 'happy2-02.png', 'happy1-00.png', 'happy1-08.png', 'happy2-15.png', 'happy1-07.png']
['sad1-18.png', 'sad2-10.png', 'sad2-01.png', 'sad2-05.png', 'sad1-00.png', 'sad2-06.png', 'sad1-12.png', 'sad1-04.png', 'sad1-19.png', 'sad1-15.png']


In [45]:
def train_happy_sad_model():

    DESIRED_ACCURACY = 0.999

    class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > DESIRED_ACCURACY):
          print("\nДостигнута точность 99.9%, обучение прерывается!")
          self.model.stop_training = True

    callbacks = myCallback()
    
    # Этот блок кода должен задать и скомпилировать модель. Пожалуйста, имейте в виду, что размер изображений 150 X 150
    model = tf.keras.models.Sequential([
      # This is the first convolution
      tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
      tf.keras.layers.MaxPooling2D(2, 2),
      # The second convolution
      tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      # The third convolution
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      # Flatten the results to feed into a DNN
      tf.keras.layers.Flatten(),
      # 512 neuron hidden layer
      tf.keras.layers.Dense(512, activation='relu'),
      # Только 1 выходной нейрон. Он будет содержать значение от 0 до 1, где 0 для 1 класса («happy») и 1 для другого («sad»)
      tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    from tensorflow.keras.optimizers import RMSprop

    model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])
        

     # Этот блок кода должен создать экземпляр ImageDataGenerator с именем train_datagen
      # и train_generator, вызовом train_datagen.flow_from_directory

    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale=1/255)

    # Используйте target_size  150 X 150.
    train_generator = train_datagen.flow_from_directory(
            '/tmp/h-or-s',
            target_size=(150, 150),
            batch_size=32,
            class_mode='binary')
    # ожидаемая строка вывода: 'Found 80 images belonging to 2 classes'

    STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size

    history = model.fit_generator(
      train_generator, 
      steps_per_epoch=STEP_SIZE_TRAIN,
      epochs=20,
      verbose=1,
      callbacks=[callbacks])
 
    # model fitting
    return history.history['accuracy'][-1]

In [46]:
# ожидаемая строка вывода: "Достигнута точность 99.9%, обучение прерывается!"
train_happy_sad_model()

Found 80 images belonging to 2 classes.
Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


2/2 [==============================] - 2s 428ms/step - loss: 2.0139 - accuracy: 0.5833
Epoch 2/20
2/2 [==============================] - 1s 714ms/step - loss: 0.8600 - accuracy: 0.4167
Epoch 3/20
2/2 [==============================] - 1s 435ms/step - loss: 0.5128 - accuracy: 0.8125
Epoch 4/20
2/2 [==============================] - 1s 715ms/step - loss: 0.3279 - accuracy: 0.8958
Epoch 5/20
2/2 [==============================] - 1s 438ms/step - loss: 0.4338 - accuracy: 0.7917
Epoch 6/20
2/2 [==============================] - 1s 411ms/step - loss: 0.3675 - accuracy: 0.8750
Epoch 7/20
2/2 [==============================] - 1s 709ms/step - loss: 0.1234 - accuracy: 0.9583
Epoch 8/20
2/2 [==============================] - 1s 430ms/step - loss: 0.1233 - accuracy: 0.9583
Epoch 9/20
2/2 [==============================] - 1s 685ms/step - loss: 0.1203 - accuracy: 0.9688
Epoch 10/20
2/2 [==============================] - 1s 433ms/step - loss: 0.0919 - accuracy: 1.0000

Достигнута точность 99.9%, об

1.0